In [52]:
# !pip install efficientnet
# !pip install pydot
# !pip install graphviz
# !pip install pydotplus
import tqdm
from tqdm import tqdm_notebook
import math
from PIL import Image
# import pydot
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow.keras.backend as K
# import pydotplus
# from pydotplus import graphviz
# import pydot
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.utils.data_utils import Sequence
# import efficientnet.tfkeras as efn
from keras.applications import VGG16
tf.keras.backend.clear_session()
# BytesList = tf.train.BytesList
# FloatList = tf.train.FloatList
# Int64List = tf.train.Int64List
# Feature = tf.train.Feature
# Features = tf.train.Features
# Example = tf.train.Example

# to make this notebook's output stable across runs
# np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)


print(f"Tensor flow version --> {tf.__version__}")
# from tensorflow.keras.utils import Sequence
print(f"Keras version --> {keras.__version__}")

PATH = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/"
OUTPUT_PATH_LUNG_MASK_IMAGES_1MM = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/ads/lung_mask_1mm/"
OUTPUT_PATH = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/ads/"
OUTPUT_PATH_CSV = "/Users/paritoshgupta/Desktop/aamlp/aamlp/input/osic-pulmonary-fibrosis-progression/ads/osic_csv_files/"


Tensor flow version --> 2.2.0
Keras version --> 2.3.0-tf


In [53]:
# path parameters
# OUTPUT_PATH = "../input/osiccsvfiles/osic_csv_files/"
# OUTPUT_PATH_LUNG_MASK_IMAGES_1MM = "../input/osic-ads-images/lung_mask_1mm/"

# augmentation parameters
ROT_ = 0
SHR_ = 2
HZOOM_ = 4
WZOOM_ = 4
HSHIFT_ = 4
WSHIFT_ = 4


# ROT_ = 0
# SHR_ = 2.0
# HZOOM_ = 8.0
# WZOOM_ = 8.0
# HSHIFT_ = 8.0
# WSHIFT_ = 8.0

# input image size
image_size = 299

# batch size
bs = 16

# FOLDS PARAMETERS
fold = 0 # which fold to run
n_folds = 4 # CSV FILE

# FINE TUNING PARAMETERS
# Number of layers to fine-tune of base model
trainable = 8
early_stopping_fine_tuning = 8

# MODEL PARAMATERS AND LEARNING RATE
EPOCHS = 100
optimizer = keras.optimizers.Adam(lr=0.00125, beta_1=0.9, beta_2=0.999)
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=1)
checkpoint_cb = keras.callbacks.ModelCheckpoint(f"/kaggle/working/osic_pat_level_image_index_{n_folds}FOLDS_{EPOCHS}E_f_{fold}.h5",
                                                save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

# AUGMENTATION PARAMETER
TTA_STEPS = 11

# Test Patients
patients_test = ["ID00419637202311204720264", "ID00421637202311550012437", "ID00422637202311677017371", "ID00423637202312137826377", "ID00426637202313170790466"]

# Junk Patients
junk_patients = [ "ID00170637202238079193844","ID00090637202204766623410"] 


########################## PLAYGROUND ########################################
# AUG_BATCH = 16
# patients_image_perc_20 = ["ID00168637202237852027833","ID00099637202206203080121", 
#                           "ID00285637202278913507108", "ID00371637202296828615743", 
#                           "ID00186637202242472088675"]

# patients_image_perc_70 = ["ID00123637202217151272140", "ID00108637202209619669361",
#                           "ID00190637202244450116191", "ID00135637202224630271439",
#                           "ID00020637202178344345685", "ID00027637202179689871102",
#                           "ID00035637202182204917484", "ID00109637202210454292264",
#                           "ID00233637202260580149633", "ID00089637202204675567570",
#                           "ID00032637202181710233084", "ID00393637202302431697467",
#                           "ID00075637202198610425520", "ID00067637202189903532242"]

# no good percentile index [0.2, 0.3, 0.7] for these 2 Patient 
#"ID00094637202205333947361", #"ID00086637202203494931510",
#"ID00067637202189903532242", "ID00014637202177757139317", \ # "ID00240637202264138860065", #  "ID00122637202216437668965"

# print(f"# of Patients with 20th Percentile Image as Input to pre-trained model --> {len(set(patients_image_perc_20))}")
# print(f"# of Patients with 70th Percentile Image as Input to pre-trained model --> {len(set(patients_image_perc_70))}")



In [54]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))


def transform(image, image_size):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = image_size[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    
    idx = tf.stack([x,y,z])    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)

    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])


def score(y_true, y_pred):
    C1 = tf.constant(70, dtype='float32')
    C2 = tf.constant(1000, dtype="float32")
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = (delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2)
    return keras.backend.mean(metric)

def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return keras.backend.mean(v)

def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss

def convert_to_tensor(arg):
    out = tf.convert_to_tensor(arg, dtype=tf.float32)
    return out

def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")    

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

def central_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]])
    top_crop = (shape[0] - min_dim) // 4
    bottom_crop = shape[0] - top_crop
    left_crop = (shape[1] - min_dim) // 4
    right_crop = shape[1] - left_crop
    return image[top_crop:bottom_crop, left_crop:right_crop]

def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def cutmix_mixup(image):
    DIM = IMAGE_SIZE[0]
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2 = cutmix(image, CUTMIX_PROB)
    image3 = mixup(image,MIXUP_PROB)
    imgs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    return image4

def cutmix(image, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    
    imgs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    return image2

def mixup(image, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    
    imgs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    return image2


def preprocess(image):

#     if aug:
#         image = tf.image.random_flip_left_right(image)
            #img = tf.image.random_hue(img, 0.01)
#         image = tf.image.random_saturation(image, 0.7, 1.3)
#         image = tf.image.random_contrast(image, 0.8, 1.2)
#         image = tf.image.random_brightness(image, 0.1)  
    resized_image = tf.image.resize(image, [image_size, image_size])
    final_image = keras.applications.xception.preprocess_input(resized_image*255)
    
    return final_image

class DataGenerator(Sequence):
    
    def __init__(self, dataset, batch_size, aug, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.aug = aug
        self.dataset = dataset
        self.shuffle = shuffle
        self.indexes = np.arange(self.dataset.shape[0])
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.dataset.shape[0]/self.batch_size))
 
 
    def __getitem__(self, index):
        # Generate indexes of the batch
        idxs = [i for i in range(index*self.batch_size,(index+1)*self.batch_size)]
 
        # Find list of IDs
        ids_tmp = [self.indexes[k] for k in idxs]
        
        # Generate data
        # tabular
        tab = self.dataset.loc[ids_tmp, ['Male', 'Female', 'Ex-smoker', 'Never smoked', 
                                         'Currently smokes', 'age', 'week', 'base_fvc']].to_numpy(dtype=np.float32)
        
        # image
        patient_ids = list(self.dataset.loc[ids_tmp, 'Patient'])
        imgs_master = []
        
        for pat_id in patient_ids:    
            images = os.listdir(os.path.join(OUTPUT_PATH_LUNG_MASK_IMAGES_1MM, pat_id))
            sorted_images = sorted(images,key=lambda x: int(os.path.splitext(x)[0]))
#             slices = [Image.open(f"{OUTPUT_PATH_LUNG_MASK_IMAGES_1MM}/{pat_id}/{sorted_images[index]}").convert('L') for index, file in enumerate(sorted_images)]    
            
            
            ######## OLD ########
#             img1 = np.mean([np.array(slices[i]) for i in range(int(0.15 * len(slices)), int(0.4 * len(slices)))], axis=0)/255
#             img1 = tf.convert_to_tensor(img1.reshape((*img1.shape, 1)))
#             img2 = np.mean([np.array(slices[i]) for i in range(int(0.4 * len(slices)/2), int(0.65 * len(slices)))], axis=0)/255
#             img2 = tf.convert_to_tensor(img2.reshape((*img2.shape, 1)))
#             img3 = np.mean([np.array(slices[i]) for i in range(int(0.65 * len(slices)), int(0.9 * len(slices)))], axis=0)/255
#             img3 = tf.convert_to_tensor(img3.reshape((*img3.shape, 1)))

#             randint = np.random.randint(0,3)
#             if randint == 0:
#                 img_to_resize = img1
#             elif randint == 1:
#                 img_to_resize = img2
#             else:
#                 img_to_resize = img3

            ######## NEW Augementation ########
            ###### All processing on GPU!!! #######
        
            ##### FOR TEST PATIENTS ONLY ######
            if pat_id in patients_test and self.aug==True:
                combo = [0.2, 0.3, 0.5, 0.7, 0.8]
                perc = np.random.choice(combo)
                print(perc)
            else:
                combo = [0.2, 0.3, 0.5, 0.7, 0.8]
                perc = np.random.choice(combo)
              
############## NEW EXPERIMENT TO BE TRIED ###################             
#             combo = [0.2, 0.3, 0.7]
#             perc = np.random.choice(combo)
                

#             if pat_id in patients_image_perc_20:
#                 perc = 0.2
#             elif pat_id in patients_image_perc_70:
#                 perc = 0.7
#             elif pat_id in "ID00423637202312137826377":
#                 print("yes")
#                 combo = [0.4, 0.3]
#                 perc = np.random.choice(combo)
#                 print(perc)
#                 perc = 0.4
#             elif pat_id in "ID00426637202313170790466":
#                 print("yes")
#                 perc = 0.7
#             elif pat_id in "ID00421637202311550012437":
#                 perc = 0.2
            
            index = int(len(sorted_images)*perc)
            image = tf.keras.preprocessing.image.img_to_array(Image.open(f"{OUTPUT_PATH_LUNG_MASK_IMAGES_1MM}/{pat_id}/{sorted_images[index]}").convert('L'))/255
            image = image[:, :, 0] 
            img_to_resize = tf.convert_to_tensor(image.reshape((*image.shape, 1)))
            images = tf.convert_to_tensor([img_to_resize]*3)
            images_reduced_mean = tf.reduce_mean(images, axis=3)
            image_pre_proc = tf.stack([images_reduced_mean[0], images_reduced_mean[1], images_reduced_mean[2]], axis=-1)

            # With Augmentation (Training time)
            if self.aug:
                trans_image = transform(image_pre_proc, image_size = image_pre_proc.shape)
                resized_image = preprocess(trans_image)[:, :, 0]
            else:
                resized_image = preprocess(image_pre_proc)[:, :, 0]
                
            out_img = tf.stack([resized_image, resized_image, resized_image], axis=-1)
            imgs_master.append(out_img)
        
#         imgs_master_cutup_mixup = cutmix_mixup(imgs_master)
        imgs_master_out = tf.stack(imgs_master)

        # target column
        FVC = self.dataset.loc[ids_tmp, 'FVC'].to_numpy(dtype=np.float32)

        # to tensor convert
        FVC = tf.convert_to_tensor(tf.constant(FVC))
        tab = tf.convert_to_tensor(tf.constant(tab))
        
        return [imgs_master_out, tab], FVC
 
    def on_epoch_end(self):
        self.indexes = np.arange(self.dataset.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
#         if self.shuffle:
#             np.random.shuffle(self.indexes)



def get_model():


    input_shape_image=(image_size,image_size,3)
    input_image = keras.layers.Input(shape=input_shape_image, name="Image_Input")

    ########## Im,age Input ##########
    # base_model = keras.applications.VGG19(weights="imagenet", include_top=False,
    #             input_tensor=keras.layers.Input(shape=input_shape_image))
    base_model = keras.applications.Xception(weights='imagenet', include_top=False, input_shape=input_shape_image)
    base_model.trainable=False
    base_model_output = base_model(input_image, training=False)
    layer_global_pooling = keras.layers.GlobalAveragePooling2D(name="pretrained_global_avg_pooling")(base_model_output)
    out_dense_1 = keras.layers.Dense(64, name ="output_dense_img_1",activation="relu")(layer_global_pooling)  
    bn_out_1 = keras.layers.BatchNormalization()(out_dense_1)
    
    
    # Monte Carlo Dropout
    dropout_img_1 = keras.layers.Dropout(0.45, name="dropout_img_1")(bn_out_1)
#     dropout_img_1 = get_dropout(bn_out_1, p=0.45, mc=mc)

    out_dense_2 = keras.layers.Dense(16, name ="output_dense_img_2",activation="relu")(dropout_img_1)  
    # out_image = keras.layers.Dense(16, name ="output_image",activation="relu")(dropout_img_2)  

    ########## Tabular Input ##########
    tabular_inp= keras.layers.Input(shape=(8,))
    # Concat - Image and Tabular
    concat = keras.layers.Concatenate(name="concat")([out_dense_2, tabular_inp])
    concat_out_1 = keras.layers.Dense(64, name="concat_out_2", activation="relu",
                                     kernel_regularizer=keras.regularizers.l2(0.1)
                                     )(concat)
    bn_out_2 = keras.layers.BatchNormalization()(concat_out_1)
    drop_out_img_tab_1 = keras.layers.Dropout(0.25, name="dropout_img_tab_2")(bn_out_2)
#     drop_out_img_tab_1 = get_dropout(bn_out_2, p=0.2, mc=mc)  
    concat_out_2 = keras.layers.Dense(32, name="concat_out_3", activation="relu")(drop_out_img_tab_1)
    p1 = keras.layers.Dense(3, activation="linear", name="p1")(concat_out_2)
    p2 = keras.layers.Dense(3, activation="relu", name="p2")(concat_out_2)
    preds = keras.layers.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                     name="preds")([p1, p2])

    model = keras.Model(inputs = [input_image, tabular_inp], outputs = preds)
    for layer in base_model.layers:
        layer.trainable = False


    print(model.summary())
    print(f" N Layers of BASE model --> {len(base_model.layers)}")
    print(f" N Layers of our model --> {len(model.layers)}")
    for index, layer in enumerate(model.layers):
        print(index, layer.name)
        
    return model 

In [55]:
model = get_model()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Image_Input (InputLayer)        [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
xception (Model)                (None, 10, 10, 2048) 20861480    Image_Input[0][0]                
__________________________________________________________________________________________________
pretrained_global_avg_pooling ( (None, 2048)         0           xception[1][0]                   
__________________________________________________________________________________________________
output_dense_img_1 (Dense)      (None, 64)           131136      pretrained_global_avg_pooling[0][
______________________________________________________________________________________________

In [79]:
model.load_weights("/Users/paritoshgupta/Downloads/osic_pat_level_image_index_4FOLDS_100E_f_0.h5")
# model2 = tf.keras.models.load_model("/Users/paritoshgupta/Downloads/osic_pat_level_image_index_10FOLDS_100E_f_0 (1).h5", compile=False)

# model.training=False
# (model.load_weights('weights.h5')

In [64]:
train = pd.read_csv(f"{PATH}/train.csv")
print(train.shape)
train_tmp1 = train.groupby(['Patient', 'Weeks']).mean().reset_index()
train_tmp2 = train[['Patient', 'Weeks', 'Sex', 'SmokingStatus']].drop_duplicates().reset_index(drop=True)
print(train_tmp1.shape)
print(train_tmp1.shape)
train = pd.merge(train_tmp1, train_tmp2, on=["Patient", "Weeks"], how="inner")
print(train.shape)
test = pd.read_csv(f"{PATH}/test.csv")
test_patients = test.Patient.unique()

print("add infos")
sub = pd.read_csv(f"{PATH}/sample_submission.csv")
sub['Patient'] = sub['Patient_Week'].apply(lambda x:x.split('_')[0])
sub['Weeks'] = sub['Patient_Week'].apply(lambda x: int(x.split('_')[-1]))
sub =  sub[['Patient','Weeks','Confidence','Patient_Week']]
sub = sub.merge(test.drop('Weeks', axis=1), on="Patient")
train['WHERE'] = 'train'
test['WHERE'] = 'val'
sub['WHERE'] = 'test'
data = train.append([test, sub])
data['min_week'] = data['Weeks']
data.loc[data.WHERE=='test','min_week'] = np.nan
data['min_week'] = data.groupby('Patient')['min_week'].transform('min')
base = data.loc[data.Weeks == data.min_week]
base = base[['Patient','FVC']].copy()
base.columns = ['Patient','min_FVC']
base['nb'] = 1
base['nb'] = base.groupby('Patient')['nb'].transform('cumsum')
base = base[base.nb==1]
base.drop('nb', axis=1, inplace=True)
data = data.merge(base, on='Patient', how='left')
data['base_week'] = data['Weeks'] - data['min_week']
del base
COLS = ['Sex','SmokingStatus'] #,'Age'
FE = ['Patient']
for col in COLS:
    for mod in data[col].unique():
        FE.append(mod)
        data[mod] = (data[col] == mod).astype(int)
#=================
data['age'] = (data['Age'] - data['Age'].min() ) / ( data['Age'].max() - data['Age'].min() )
data['base_fvc'] = (data['min_FVC'] - data['min_FVC'].min() ) / ( data['min_FVC'].max() - data['min_FVC'].min() )
data['week'] = (data['base_week'] - data['base_week'].min() ) / ( data['base_week'].max() - data['base_week'].min() )
data['percent'] = (data['Percent'] - data['Percent'].min() ) / ( data['Percent'].max() - data['Percent'].min() )
FE += ['age', 'week','base_fvc', 'FVC']
print(data.columns)
data
# print(FE)
# data[FE]

(1549, 7)
(1542, 5)
(1542, 5)
(1542, 7)
add infos
Index(['Patient', 'Weeks', 'FVC', 'Percent', 'Age', 'Sex', 'SmokingStatus',
       'WHERE', 'Confidence', 'Patient_Week', 'min_week', 'min_FVC',
       'base_week', 'Male', 'Female', 'Ex-smoker', 'Never smoked',
       'Currently smokes', 'age', 'base_fvc', 'week', 'percent'],
      dtype='object')


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,Patient_Week,...,base_week,Male,Female,Ex-smoker,Never smoked,Currently smokes,age,base_fvc,week,percent
0,ID00007637202177411956430,-4,2315.0,58.253649,79.0,Male,Ex-smoker,train,NaN,NaN,...,0.0,1,0,1,0,0,0.769231,0.241456,0.179012,0.236393
1,ID00007637202177411956430,5,2214.0,55.712129,79.0,Male,Ex-smoker,train,NaN,NaN,...,9.0,1,0,1,0,0,0.769231,0.241456,0.234568,0.215941
2,ID00007637202177411956430,7,2061.0,51.862104,79.0,Male,Ex-smoker,train,NaN,NaN,...,11.0,1,0,1,0,0,0.769231,0.241456,0.246914,0.184960
3,ID00007637202177411956430,9,2144.0,53.950679,79.0,Male,Ex-smoker,train,NaN,NaN,...,13.0,1,0,1,0,0,0.769231,0.241456,0.259259,0.201767
4,ID00007637202177411956430,11,2069.0,52.063412,79.0,Male,Ex-smoker,train,NaN,NaN,...,15.0,1,0,1,0,0,0.769231,0.241456,0.271605,0.186580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,ID00426637202313170790466,129,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_129,...,129.0,1,0,0,1,0,0.615385,0.354755,0.975309,0.345604
2273,ID00426637202313170790466,130,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_130,...,130.0,1,0,0,1,0,0.615385,0.354755,0.981481,0.345604
2274,ID00426637202313170790466,131,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_131,...,131.0,1,0,0,1,0,0.615385,0.354755,0.987654,0.345604
2275,ID00426637202313170790466,132,2925.0,71.824968,73.0,Male,Never smoked,test,100.0,ID00426637202313170790466_132,...,132.0,1,0,0,1,0,0.615385,0.354755,0.993827,0.345604


In [71]:
df_test = data.loc[data.WHERE=='train']
df_test = df_test[df_test.Patient.isin(test_patients)].reset_index(drop=True)
df_sub = data.loc[data.WHERE=='test'].reset_index(drop=True)
# df_sub.head()
# df_sub.shape
# df_test.columns == df_sub.columns

In [72]:
test_pred = model.predict(DataGenerator(dataset=df_test, batch_size=4, aug=False, shuffle=False))
df_test['confidence'] = test_pred[:, 2] - test_pred[:, 0]
df_test['pred_fvc'] = test_pred[:, 1]
df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

print(f"# Test Data shape --> {df_test.shape}")
print(f"# Predictions shape --> {test_pred.shape}")
print(f"\n**Overall Mean Score WITHOUT Augmentation on BASE FREEZED + CUSTOM MODEL --> {np.mean(df_test.score)} *****")
print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")
df_test.sample(frac=1).head(10)

# Test Data shape --> (44, 28)
# Predictions shape --> (44, 3)

**Overall Mean Score WITHOUT Augmentation on BASE FREEZED + CUSTOM MODEL --> -7.111129832585889 *****

**Patient level Mean score -->
Patient
ID00419637202311204720264   -7.071284
ID00421637202311550012437   -7.127086
ID00422637202311677017371   -6.570173
ID00423637202312137826377   -7.681104
ID00426637202313170790466   -7.045895
Name: score, dtype: float64


,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,WHERE,Confidence,Patient_Week,...,age,base_fvc,week,percent,confidence,pred_fvc,sigma_clipped,diff,delta,score
23,ID00422637202311677017371,23,1946.0,77.308120,73.0,Male,Ex-smoker,train,NaN,NaN,...,0.615385,0.169948,0.283951,0.389727,443.011841,1884.736816,443.011841,61.263184,61.263184,-6.635739
35,ID00426637202313170790466,0,2925.0,71.824968,73.0,Male,Never smoked,train,NaN,NaN,...,0.615385,0.354755,0.179012,0.345604,678.355225,2917.615723,678.355225,7.384277,7.384277,-6.881639
15,ID00421637202311550012437,41,2833.0,84.861011,68.0,Male,Ex-smoker,train,NaN,NaN,...,0.487179,0.320208,0.339506,0.450506,600.267578,2575.913086,600.267578,257.086914,257.086914,-7.349639
34,ID00423637202312137826377,70,2578.0,62.030799,72.0,Male,Ex-smoker,train,NaN,NaN,...,0.589744,0.423291,0.506173,0.266789,690.494385,2972.111572,690.494385,394.111572,394.111572,-7.691168
5,ID00419637202311204720264,18,2694.0,62.610393,73.0,Male,Ex-smoker,train,NaN,NaN,...,0.615385,0.372400,0.253086,0.271453,669.584717,2880.650391,669.584717,186.650391,186.650391,-7.247451
32,ID00423637202312137826377,42,2516.0,60.538980,72.0,Male,Ex-smoker,train,NaN,NaN,...,0.589744,0.423291,0.333333,0.254784,711.332764,3064.757324,711.332764,548.757324,548.757324,-8.004708
13,ID00421637202311550012437,23,2853.0,85.460101,68.0,Male,Ex-smoker,train,NaN,NaN,...,0.487179,0.320208,0.228395,0.455327,615.672852,2643.499023,615.672852,209.500977,209.500977,-7.250518
7,ID00419637202311204720264,43,2793.0,64.911221,73.0,Male,Ex-smoker,train,NaN,NaN,...,0.615385,0.372400,0.407407,0.289968,648.634521,2788.323242,648.634521,4.676758,4.676758,-6.831640
42,ID00426637202313170790466,43,2975.0,73.052745,73.0,Male,Never smoked,train,NaN,NaN,...,0.615385,0.354755,0.444444,0.355484,642.492432,2759.395264,642.492432,215.604736,215.604736,-7.286504
18,ID00422637202311677017371,6,1930.0,76.672493,73.0,Male,Ex-smoker,train,NaN,NaN,...,0.615385,0.169948,0.179012,0.384612,458.342041,1951.798462,458.342041,21.798462,21.798462,-6.541448


In [73]:
sub_pred = model.predict(DataGenerator(dataset=df_sub, batch_size=10, aug=False, shuffle=False))
df_sub['confidence'] = sub_pred[:, 2] - sub_pred[:, 0]
df_sub['predicted_fvc'] = sub_pred[:, 1]


In [74]:
# df_sub.head()
out = df_sub[['Patient_Week', 'confidence', 'predicted_fvc']]
out = out.rename(columns={'confidence': 'Confidence', 'predicted_fvc': 'FVC'})
out[['Patient_Week', 'FVC', 'Confidence']]



,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2975.199951,690.763184
1,ID00419637202311204720264_-11,2990.359375,694.513428
2,ID00419637202311204720264_-10,2986.666504,693.675293
3,ID00419637202311204720264_-9,2982.973145,692.837158
4,ID00419637202311204720264_-8,2979.280029,691.999268
...,...,...,...
725,ID00426637202313170790466_129,2423.314941,565.943848
726,ID00426637202313170790466_130,2437.514648,569.415039
727,ID00426637202313170790466_131,2415.928955,564.267578
728,ID00426637202313170790466_132,2430.128418,567.739014


In [39]:
df_out['Patient_Week'] = df_out['Patient'].astype(str) + "_" + df_out['Weeks'].astype(str)
df_out
df_out = df_out.rename(columns={'confidence': 'Confidence', 'predicted_fvc': 'FVC'})
out = df_out.drop(['week', 'Patient', 'Weeks'], axis=1)[['Patient_Week', 'FVC', 'Confidence']]
out.shape

,Patient_Week,FVC,Confidence
0,ID00419637202311204720264_-12,2986.336426,693.460205
1,ID00419637202311204720264_-11,2987.749756,693.886719
2,ID00419637202311204720264_-10,2967.813965,689.087158
3,ID00419637202311204720264_-9,2964.120605,688.248779
4,ID00419637202311204720264_-8,2971.563965,690.107666
...,...,...,...
725,ID00426637202313170790466_129,2423.314941,565.943848
726,ID00426637202313170790466_130,2423.977051,566.193359
727,ID00426637202313170790466_131,2420.283936,565.355469
728,ID00426637202313170790466_132,2416.590820,564.517090


In [105]:
# test_pred = model.predict(DataGenerator(dataset=df_test, batch_size=4, aug=False, shuffle=False))
# df_test['confidence'] = test_pred[:, 2] - test_pred[:, 0]
# df_test['pred_fvc'] = test_pred[:, 1]
# df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
# df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
# df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
# df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

# print(f"# Test Data shape --> {df_test.shape}")
# print(f"# Predictions shape --> {test_pred.shape}")
# print(f"\n**Overall Mean Score WITHOUT Augmentation on BASE FREEZED + CUSTOM MODEL --> {np.mean(df_test.score)} *****")
# print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")
# df_test.sample(frac=1).head(10)

# Test Data shape --> (44, 16)
# Predictions shape --> (44, 3)

**Overall Mean Score WITHOUT Augmentation on BASE FREEZED + CUSTOM MODEL --> -7.110570220473196 *****

**Patient level Mean score -->
Patient
ID00419637202311204720264   -7.073377
ID00421637202311550012437   -7.125085
ID00422637202311677017371   -6.562646
ID00423637202312137826377   -7.684095
ID00426637202313170790466   -7.046767
Name: score, dtype: float64


,Patient,Male,Female,Ex-smoker,Never smoked,Currently smokes,age,week,base_fvc,FVC,confidence,pred_fvc,sigma_clipped,diff,delta,score
0,ID00419637202311204720264,1,0,1,0,0,0.615385,0.179012,0.372400,3020.0,678.376221,2919.860840,678.376221,100.139160,100.139160,-7.075036
40,ID00426637202313170790466,1,0,0,1,0,0.615385,0.296296,0.354755,2978.0,658.123779,2829.554199,658.123779,148.445801,148.445801,-7.154955
10,ID00421637202311550012437,1,0,1,0,0,0.487179,0.191358,0.320208,2756.0,622.527100,2673.339600,622.527100,82.660400,82.660400,-6.968143
37,ID00426637202313170790466,1,0,0,1,0,0.615385,0.234568,0.354755,2916.0,667.591797,2870.840332,667.591797,45.159668,45.159668,-6.945916
1,ID00419637202311204720264,1,0,1,0,0,0.615385,0.185185,0.372400,2859.0,678.802490,2921.274170,678.802490,62.274170,62.274170,-6.996645
41,ID00426637202313170790466,1,0,0,1,0,0.615385,0.370370,0.354755,2908.0,652.376953,2803.130127,652.376953,104.869873,104.869873,-7.054532
14,ID00421637202311550012437,1,0,1,0,0,0.487179,0.265432,0.320208,2716.0,610.779053,2622.139648,610.779053,93.860352,93.860352,-6.978636
20,ID00422637202311677017371,1,0,1,0,0,0.615385,0.222222,0.169948,1955.0,454.649170,1935.056885,454.649170,19.943115,19.943115,-6.528134
8,ID00419637202311204720264,1,0,1,0,0,0.615385,0.506173,0.372400,2727.0,635.853516,2731.843750,635.853516,4.843750,4.843750,-6.812315
4,ID00419637202311204720264,1,0,1,0,0,0.615385,0.222222,0.372400,2738.0,674.401367,2901.725098,674.401367,163.725098,163.725098,-7.203729


In [75]:
TTA_STEPS = 11

In [76]:
test_pred_aug = np.stack([model.predict(DataGenerator(dataset=df_test, batch_size=4, aug=True, shuffle=False)) for sample in range(TTA_STEPS)])
test_pred_mean = test_pred_aug.mean(axis=0)
df_test['confidence'] = test_pred_mean[:, 2] - test_pred_mean[:, 0]
df_test['pred_fvc'] = test_pred_mean[:, 1]
df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

print(f"# Test Data shape --> {df_test.shape}")
print(f"# Predictions shape --> {test_pred_aug.shape}")
print(f"\n**Overall Mean Score WITH Augmentation ON FINE TUNED MODEL--> {np.mean(df_test.score)} *****")
print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")


0.7
0.5
0.5
0.2
0.3
0.5
0.8
0.8
0.7
0.7
0.5
0.8
0.3
0.7
0.3
0.3
0.5
0.8
0.2
0.8
0.8
0.2
0.8
0.7
0.5
0.8
0.7
0.7
0.5
0.7
0.7
0.2
0.8
0.5
0.5
0.7
0.3
0.3
0.2
0.5
0.5
0.5
0.8
0.5
0.8
0.7
0.3
0.7
0.2
0.7
0.5
0.5
0.8
0.7
0.8
0.8
0.7
0.5
0.5
0.5
0.5
0.8
0.7
0.8
0.2
0.7
0.7
0.7
0.2
0.5
0.8
0.3
0.8
0.7
0.3
0.5
0.8
0.7
0.2
0.8
0.5
0.3
0.3
0.2
0.3
0.7
0.7
0.3
0.3
0.7
0.3
0.3
0.5
0.2
0.3
0.2
0.3
0.7
0.8
0.5
0.8
0.8
0.8
0.8
0.5
0.2
0.3
0.8
0.5
0.2
0.8
0.3
0.7
0.7
0.5
0.7
0.5
0.2
0.5
0.5
0.8
0.7
0.7
0.2
0.5
0.8
0.7
0.8
0.7
0.2
0.8
0.3
0.3
0.5
0.7
0.8
0.7
0.7
0.3
0.5
0.5
0.5
0.7
0.2
0.8
0.7
0.7
0.2
0.3
0.8
0.2
0.3
0.8
0.2
0.3
0.3
0.8
0.2
0.5
0.2
0.2
0.5
0.8
0.3
0.7
0.7
0.2
0.7
0.2
0.3
0.5
0.2
0.2
0.7
0.3
0.3
0.5
0.5
0.2
0.8
0.8
0.7
0.8
0.5
0.7
0.2
0.3
0.7
0.3
0.7
0.8
0.8
0.2
0.5
0.5
0.7
0.5
0.7
0.8
0.3
0.8
0.5
0.2
0.7
0.7
0.8
0.3
0.3
0.5
0.3
0.3
0.3
0.3
0.8
0.7
0.3
0.2
0.5
0.2
0.7
0.2
0.2
0.7
0.8
0.7
0.2
0.5
0.2
0.8
0.5
0.8
0.5
0.5
0.2
0.8
0.3
0.3
0.3
0.2
0.8
0.2
0.3
0.8
0.8
0.2
0.5
0.8
0.2
0.7
0.5


In [107]:
df_test['confidence'] = test_pred[:, 2] - test_pred[:, 0]
df_test['pred_fvc'] = 0.996 * test_pred[:, 1]
df_test['sigma_clipped'] = df_test['confidence'].apply(lambda x: max(x, 70))
df_test['diff'] = abs(df_test['FVC'] - df_test['pred_fvc'])
df_test['delta'] = df_test['diff'].apply(lambda x: min(x, 1000))
df_test['score'] = -math.sqrt(2)*df_test['delta']/df_test['sigma_clipped'] - np.log(math.sqrt(2)*df_test['sigma_clipped'])

print(f"# Test Data shape --> {df_test.shape}")
print(f"# Predictions shape --> {test_pred_aug.shape}")
print(f"\n**Overall Mean Score WITH Augmentation ON FINE TUNED MODEL--> {np.mean(df_test.score)} *****")
print(f"\n**Patient level Mean score -->\n{df_test.groupby('Patient')['score'].mean()}")


print(np.mean(df_test.score))
df_test.groupby('Patient')['score'].mean()

# Test Data shape --> (44, 16)
# Predictions shape --> (9, 44, 3)

**Overall Mean Score WITH Augmentation ON FINE TUNED MODEL--> -7.11232963277107 *****

**Patient level Mean score -->
Patient
ID00419637202311204720264   -7.062865
ID00421637202311550012437   -7.149367
ID00422637202311677017371   -6.556884
ID00423637202312137826377   -7.665138
ID00426637202313170790466   -7.065678
Name: score, dtype: float64
-7.11232963277107


Patient
ID00419637202311204720264   -7.062865
ID00421637202311550012437   -7.149367
ID00422637202311677017371   -6.556884
ID00423637202312137826377   -7.665138
ID00426637202313170790466   -7.065678
Name: score, dtype: float64

In [87]:
df_tmp = pd.merge(df_test, df_sub_mapping, on =["Patient", "week"], how="inner")
print(df_tmp.shape)
print(df_tmp.columns)
df_tmp.head()

(44, 17)
Index(['Patient', 'Male', 'Female', 'Ex-smoker', 'Never smoked',
       'Currently smokes', 'age', 'week', 'base_fvc', 'FVC', 'confidence',
       'pred_fvc', 'sigma_clipped', 'diff', 'delta', 'score', 'Weeks'],
      dtype='object')


,Patient,Male,Female,Ex-smoker,Never smoked,Currently smokes,age,week,base_fvc,FVC,confidence,pred_fvc,sigma_clipped,diff,delta,score,Weeks
0,ID00419637202311204720264,1,0,1,0,0,0.615385,0.179012,0.3724,3020.0,678.304443,2919.496094,678.304443,100.503906,100.503906,-7.075713,6
1,ID00419637202311204720264,1,0,1,0,0,0.615385,0.185185,0.3724,2859.0,677.103027,2914.316406,677.103027,55.316406,55.316406,-6.979932,7
2,ID00419637202311204720264,1,0,1,0,0,0.615385,0.197531,0.3724,2783.0,674.971436,2905.002930,674.971436,122.002930,122.002930,-7.116867,9
3,ID00419637202311204720264,1,0,1,0,0,0.615385,0.203704,0.3724,2719.0,674.722168,2903.796387,674.722168,184.796387,184.796387,-7.248207,10
4,ID00419637202311204720264,1,0,1,0,0,0.615385,0.222222,0.3724,2738.0,672.301025,2893.079102,672.301025,155.079102,155.079102,-7.183495,13


In [90]:
l = df_tmp[['Patient', 'Weeks', 'FVC', 'confidence', 'pred_fvc', 'score']]

In [95]:
l1 = l.groupby(['Patient']).tail(3).reset_index(drop=True)

In [98]:
l1.groupby(['Patient'])['score'].mean()

Patient
ID00419637202311204720264   -6.913969
ID00421637202311550012437   -7.282977
ID00422637202311677017371   -6.592424
ID00423637202312137826377   -7.931163
ID00426637202313170790466   -7.120169
Name: score, dtype: float64

In [99]:
l1['score'].mean()

-7.168140479728889

In [101]:
np.std(df_test['confidence'])

91.74976

In [1]:
pd.merge()




NameError: name 'pd' is not defined

In [2]:
import pandas as pd
import numpy as np

In [19]:
df_v1 = pd.read_csv("/Users/paritoshgupta/Downloads/submission (1).csv")
df_v1.columns = ["id", "label_with_aug"]
print(df_v1.shape)
df_v2 = pd.read_csv("/Users/paritoshgupta/Downloads/submission (2).csv")
df_v2.columns = ["id", "label_without_aug"]
print(df_v2.shape)

df_v3 = pd.read_csv("/Users/paritoshgupta/Downloads/submission (3).csv")
df_v3.columns = ["id", "label_old"]

df_v4 = pd.read_csv("/Users/paritoshgupta/Downloads/submission (5).csv")
df_v4.columns = ["id", "label_with_aug_new"]


(545, 2)
(545, 2)


In [20]:
df = pd.merge(df_v1, df_v2, how="inner", on="id")
df = pd.merge(df, df_v3, how="inner", on="id")
df = pd.merge(df, df_v4, how="inner", on="id")

In [21]:
df.head(10)

,id,label_with_aug,label_without_aug,label_old,label_with_aug_new
0,df06fad17bc3_negative_exam_for_pe,0.500000,0.500000,0.785408,0.500000
1,c8039e7f9e63_negative_exam_for_pe,0.997232,0.997532,0.725287,0.997429
2,761f6f1a9f5b_negative_exam_for_pe,0.996797,0.996422,0.655640,0.996541
3,c8db5b1f6b56_negative_exam_for_pe,0.500000,0.500000,0.578280,0.500000
4,462e805da1f1_negative_exam_for_pe,0.934071,0.930760,0.811389,0.930760
5,df06fad17bc3_rv_lv_ratio_gte_1,0.395053,0.421869,0.062426,0.406557
6,c8039e7f9e63_rv_lv_ratio_gte_1,0.038783,0.031402,0.055710,0.034591
7,761f6f1a9f5b_rv_lv_ratio_gte_1,0.037782,0.036753,0.086802,0.038258
8,c8db5b1f6b56_rv_lv_ratio_gte_1,0.298062,0.301546,0.152792,0.301404
9,462e805da1f1_rv_lv_ratio_gte_1,0.109422,0.109967,0.040250,0.109967


In [22]:
all(df.label_with_aug == df.label_with_aug_new)

False